<a href="https://colab.research.google.com/github/Habsie/Velib-et-Grand-Paris-Express/blob/main/stations_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Préparation des stations du reseau ferré en idf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import plotly.express as px
import matplotlib as plt

# Import des fichiers

In [ ]:
stations_idf = pd.read_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/emplacement-des-gares-idf.csv',sep=';')

In [ ]:
zones_arrets = pd.read_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/zones-d-arrets.csv',sep=';')

In [ ]:
stations_velib = pd.read_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/stations-velib.csv')

# Traitement

## Traitement stations_idf

In [ ]:
#trie des colonnes utiles
stations_idf = stations_idf[['gares_id','Geo Point','nom','ligne','fer','id_ref_zdl']]

In [ ]:
stations_idf = stations_idf.loc[stations_idf['fer'] == 0]

In [ ]:
stations_idf['id_ref_zdl'] = stations_idf['id_ref_zdl'].astype(int)

In [ ]:
#split des coordonnées
stations_idf[['latitude','longitude']] = stations_idf['Geo Point'].str.split(',', expand=True)

In [ ]:
stations_idf['latitude'] = pd.to_numeric(stations_idf['latitude'])
stations_idf['longitude'] = pd.to_numeric(stations_idf['longitude'])

Ajout des villes dans stations_idf

In [ ]:
zones_arrets = zones_arrets[['ZdAId','ZdATown','ZdAPostalRegion']]

In [ ]:
stations_idf = stations_idf.merge(zones_arrets,left_on='id_ref_zdl',right_on='ZdAId',how='left')

## Fonction calcul 

In [ ]:
import geopy.distance
import math

def stations_rayon_300m(lat1,lon1):
#initialisation à chaque appel de la fonction
  nb_stations = 0
  capacity_tot = 0

  for station in range(len(stations_velib)):
  
    lat2 = stations_velib.iloc[station,3]
    lon2 = stations_velib.iloc[station,4]
    capacity = stations_velib.iloc[station,2]

    

    coords_1 = (lat1,lon1)
    coords_2 = (lat2,lon2)

    dist = geopy.distance.distance(coords_1, coords_2).m
    
    if dist < 300:
      nb_stations = nb_stations + 1
      capacity_tot = capacity_tot + capacity

  return nb_stations,capacity_tot
  


## Création des colonnes 'nb_stations_velib_200m' et 'nb_bornes_200m'

In [ ]:
stations_idf['result'] = stations_idf.apply( lambda stations_idf: stations_rayon_300m(stations_idf['latitude'],stations_idf['longitude']),axis=1)

In [ ]:
stations_idf['result'] = stations_idf['result'].astype(str)
stations_idf['result'] = stations_idf['result'].str.replace('(','')
stations_idf['result'] = stations_idf['result'].str.replace(')','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
stations_idf[['nb_stations_velib_300m','nb_bornes_300m']] = stations_idf['result'].str.split(',',expand=True)

In [ ]:
stations_idf['nb_stations_velib_300m'] = pd.to_numeric(stations_idf['nb_stations_velib_300m'])
stations_idf['nb_bornes_300m'] = pd.to_numeric(stations_idf['nb_bornes_300m'])

# Remplacer les arrondissements de Paris par Paris

In [ ]:
stations_idf.loc[stations_idf['ZdATown'].str.startswith('Paris'), "ZdATown"] = 'Paris'

In [ ]:
stations_idf['ZdAPostalRegion'] = stations_idf['ZdAPostalRegion'].astype(str)

In [ ]:
stations_idf.loc[stations_idf['ZdAPostalRegion'].str.startswith('75'), "ZdAPostalRegion"] = '75056'

# Nettoyage de stations_idf

In [ ]:
stations_idf = stations_idf.drop(['id_ref_zdl','ZdAId','result'],axis=1)

In [ ]:
stations_idf = stations_idf.rename(columns={"gares_id": "id gare", "ZdATown": "Commune", "ZdAPostalRegion":"Code Commune"})

In [ ]:
stations_idf

,id gare,Geo Point,nom,ligne,fer,latitude,longitude,Commune,Code Commune,nb_stations_velib_300m,nb_bornes_300m
0,19.0,"48.75494652461008,2.3009868557349513",Antony,ORLYVAL,0.0,48.754947,2.300987,Antony,92002,0,0
1,152.0,"48.87514981880684,2.295904904713175",Charles De Gaulle-Étoile,METRO 1,0.0,48.875150,2.295905,Paris,75056,4,116
2,345.0,"48.87371340578697,2.3276954096301736",Havre-Caumartin,METRO 9,0.0,48.873713,2.327695,Paris,75056,6,174
3,586.0,"48.84894464414299,2.395879435185689",Nation,METRO 9,0.0,48.848945,2.395879,Paris,75056,5,201
4,776.0,"48.85316798396978,2.343285978923924",Saint-Michel,METRO 4,0.0,48.853168,2.343286,Paris,75056,5,167
...,...,...,...,...,...,...,...,...,...,...,...
627,269.0,"48.88813847669144,2.2497927690097654",Esplanade de la Défense,METRO 1,0.0,48.888138,2.249793,Courbevoie,92026,0,0
628,1076.0,"48.93499415965736,2.3269510437797356",La Noue,TRAM 1,0.0,48.934994,2.326951,Villeneuve-la-Garenne,92078,0,0
629,1093.0,"48.87209702901093,2.4086200551769896",Adrienne Bolland,TRAM 3b,0.0,48.872097,2.408620,Paris,75056,1,33
630,1213.0,"48.896308469776066,2.3188044013442384",Honoré de Balzac,TRAM 3b,0.0,48.896308,2.318804,Paris,75056,3,109


# Extraction

In [ ]:
stations_idf.to_csv('/content/drive/MyDrive/LePont - Capstone #3/Notebook & Data/Data/stations-idf.csv', index = False)